In [5]:
import cv2
import pickle
import os
import math
import keras
import pickle
from sklearn.utils import shuffle

In [4]:
class utils:
      
    @classmethod
    def convert2RGB(cls , data):
        for index , img in enumerate(data):
            data[index] = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        return data
        
        
    @classmethod
    def __getTrainingData(cls , length , classPath,className , img_size):
        fullImageArr = list()
        imageLabels = []
        for img in os.listdir(classPath)[0:length]:
            imageLabels.append(className);
            img_arr = cv2.imread(os.path.join(classPath,img))
            img_arr = cv2.resize(img_arr , (img_size,img_size))
            
            fullImageArr.append(img_arr)
        return fullImageArr , imageLabels;
    @classmethod
    def __getDevData(cls, trainend , testend , classPath ,className , img_size):
        fullImageArr = list()
        imageLabels = []
        for img in os.listdir(classPath)[trainend :trainend+testend]:
            imageLabels.append(className)
            img_arr = cv2.imread(os.path.join(classPath,img))
            img_arr = cv2.resize(img_arr , (img_size,img_size))
            
            fullImageArr.append(img_arr)
        return fullImageArr , imageLabels

    @classmethod
    def __getTestData(cls, devend , testend , classPath ,className , img_size):
        fullImageArr = list()
        imageLabels = []
        for img in os.listdir(classPath)[devend :devend+testend]:
            imageLabels.append(className)
            img_arr = cv2.imread(os.path.join(classPath,img))
            img_arr = cv2.resize(img_arr , (img_size,img_size))
            
            fullImageArr.append(img_arr)
        return fullImageArr , imageLabels

    @classmethod
    def __getCurrentClassData(cls, classPath , className , trainPer , batchsize , img_size):
        if(batchsize):
            totalImages = len(os.listdir(classPath)[0:batchsize])
        else:
            totalImages = len(os.listdir(classPath))

        trainingDataLen = math.floor((totalImages/100) * trainPer)
        LenLeft = totalImages - trainingDataLen;
        DevDataLen = math.floor(LenLeft / 2); 
        testDataLen = LenLeft - DevDataLen;
        
        
        X_trainingData , Y_train_labels= cls.__getTrainingData(trainingDataLen , classPath,className , 
                                                                img_size)
        X_devData , Y_dev_labels = cls.__getDevData(trainingDataLen , DevDataLen , classPath , className ,
                                                     img_size)
        X_testData , Y_test_labels= cls.__getTestData(trainingDataLen + DevDataLen , testDataLen , classPath,
                                                       className , img_size)

        return X_trainingData , Y_train_labels , X_devData , Y_dev_labels , X_testData ,Y_test_labels
    
    @classmethod
    def load_data(cls,totalClasses , img_size=50 , data_path='../data/input' , batch_size=False, trainPer=80):
        classesList = os.listdir(data_path)
        X_TrainingData = []
        X_TestData = []
        Y_TrainingData = []
        Y_TestData = []
        X_DevData = []
        Y_DevData = []
        for index , clss in enumerate(classesList):
            className = index
            print(className)
            X_train , Y_train , X_dev ,Y_dev , X_test , Y_test = cls.__getCurrentClassData(
                                                                        os.path.join(data_path , clss), 
                                                                        className , trainPer , 
                                                                        batch_size ,img_size);
            
            
            X_TrainingData += X_train
            X_TestData += X_test
            Y_TrainingData += Y_train
            Y_TestData += Y_test
            X_DevData += X_dev
            Y_DevData += Y_dev

        X_TrainingData = np.array(X_TrainingData)
        Y_TrainingData = np.array(Y_TrainingData)
        
        X_TestData = np.array(X_TestData)
        Y_TestData = np.array(Y_TestData)
        
        X_DevData = np.array(X_DevData)
        Y_DevData = np.array(Y_DevData)
        
        print(X_TrainingData.shape)
        print(Y_TrainingData.shape)

        print(X_TestData.shape)
        print(Y_TestData.shape)

        print(X_DevData.shape)
        print(Y_DevData.shape)
        X_TrainingData , Y_TrainingData = shuffle(X_TrainingData , Y_TrainingData) 
        X_TestData , Y_TestData = shuffle(X_TestData ,Y_TestData )
        X_DevData , Y_DevData  = shuffle(X_DevData , Y_DevData)
        
        return X_TrainingData , X_TestData , Y_TrainingData , Y_TestData , X_DevData , Y_DevData
    
    @classmethod
    def save_data(cls , fileName , dataToStore , pathToStore = '../data/working/'):
        with open(os.path.join(pathToStore, fileName) , 'wb') as file:
            pickle.dump(dataToStore , file)
    @classmethod
    def load_saved_data(cls , 
                        fileNames=['X_Train' , 'X_Test' , 'X_Dev', 'Y_Train' , 'Y_Test' , 'Y_Dev'] ,
                        pathToStore = '../data/working/'):
        data = []
        for fileName in fileNames:
            with open(os.path.join(pathToStore , fileName) , 'rb') as file:
                data.append(pickle.load(file))
        
        return data
    
    @classmethod
    def ifDataExits(cls , fileName , whereToCheck = '../data/working/'):
        exists = os.path.isfile(os.path.join(whereToCheck , fileName))
        
        if exists:
            return True
        else:
            return False
    